In [4]:
import json
import numpy as np
import requests as r
import pandas as pd
from datetime import date
from datetime import datetime

code_list = ["HSI","0005","0011","0388","0939","1299","1398","2318","2388","2628","3328","3988"]
url_list = [f'%5EHSI', '0005.HK', '0011.HK', '0388.HK', '0939.HK', '1299.HK', '1398.HK', '2318.HK', '2388.HK', '2628.HK', '3328.HK', '3988.HK']
day_b10 = int(datetime(int(date.today().strftime("%Y"))-10,int(date.today().strftime("%m")),int(date.today().strftime(f"%d"))).timestamp())
today = int(datetime(int(date.today().strftime("%Y")),int(date.today().strftime("%m")),int(date.today().strftime(f"%d"))+1).timestamp())

for i in range(len(code_list)) :
        url = (f"https://query1.finance.yahoo.com/v8/finance/chart/{url_list[i]}?symbol={url_list[i]}&period1={day_b10}&period2={today}&useYfid=true&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=c0iNuYsQEnn&corsDomain=finance.yahoo.com")
        header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}
        res = r.get(url,headers=header)
        content = json.loads(res.text)

        date1 = content['chart']['result'][0]['timestamp']
        date2 = [date.fromtimestamp(date1[j]).strftime(f"%Y-%m-%d") for j in range(len(date1))]
        open = content['chart']['result'][0]['indicators']['quote'][0]['open']
        close = content['chart']['result'][0]['indicators']['quote'][0]['close']
        adjclose = content['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']
        high = content['chart']['result'][0]['indicators']['quote'][0]['high']
        low = content['chart']['result'][0]['indicators']['quote'][0]['low']
        volume = content['chart']['result'][0]['indicators']['quote'][0]['volume']

        ###Count the day of "close" and "adjclose" are different.
        #n = 0
        #nl = []
        #for k in range(len(close)) :
        #    if close[k] == adjclose[k]:
        #        continue
        #    else :
        #       n += 1
        #       nl.append(date2[k])
        #if n != 0 :
        #        print(f'''There are {n} days data difference between "close" and "adjclose" in {co_list[i]}.''')

        d = {"date": date2, "open": open, "close": close, "high": high, "low": low, "volume": volume}
        df = pd.DataFrame(d)[::-1]

        rate1 = (df.loc[:,"close"].values[:-1]-df.loc[:,"close"].values[1:])/df.loc[:,"close"].values[1:]*100
        rate2 = np.array([0])
        mix_rate = np.append(rate1,rate2)
        df["rate%"] = mix_rate.tolist()
        df["difference"] = df["high"]-df["low"]

        df.drop(df[df['close'].isnull()].index, inplace=True)
        df.drop(df[(df['volume']==0)&(df['rate%']==0)].index, inplace=True)

        x = df.pop("rate%")
        df.insert(3,"rate%",x)
        x = df.pop("difference")
        df.insert(6,"difference",x)

        if i == 0 :
                df_HSI = df
        elif i == 1 :
                df_0005 = df
        elif i == 2 :
                df_0011 = df
        elif i == 3 :
                df_0388 = df
        elif i == 4 :
                df_0939 = df
        elif i == 5 :
                df_1299 = df
        elif i == 6 :
                df_1398 = df
        elif i == 7 :
                df_2318 = df
        elif i == 8 :
                df_2388 = df
        elif i == 9 :
                df_2628 = df
        elif i == 10 :
                df_3328 = df
        elif i == 11 :
                df_3988 = df
        else :
                print("Check the code.")



name_list = ["Hang Seng Index","HSBC Holdings plc","Hang Seng Bank Ltd","HKEx Limited","China Construction Bank","AIA Group Limited","Industrial and Commercial Bank of China","Ping An Insurance","BOC Hong Kong (Holdings) Ltd","China Life","Bank of Communications Ltd","Bank of China Ltd"]
df_list = [df_HSI,df_0005,df_0011,df_0388,df_0939,df_1299,df_1398,df_2318,df_2388,df_2628,df_3328,df_3988]

def save(code) :
    today = str(date.today().strftime("%Y"))+str(date.today().strftime("%m"))+str(date.today().strftime(f"%d"))
    if code == "all" :
        for i in code_list :
            df_list[code_list.index(i)].to_csv(f"{i}_{today}.csv")
        print("HSI and 11 of it's financials industry sectors stock csv file are saved successfully.")
    elif code not in code_list :
        df_error = pd.DataFrame(list(zip(name_list,code_list)),columns=["Name","code"])
        print(f'''Please type "all" for saving all data frame or input a code from dictionary as below :\n{df_error}''')
    else :
        df_list[code_list.index(code)].to_csv(f"{code}_{today}.csv")
        print(f'''"{code}" stock csv file is saved successfully.''')

In [7]:
save('all')

HSI and 11 of it's financials industry sectors stock csv file are saved successfully.
